# Environment Setting

In [1]:
!pip install 'Fiona==1.8.18'
!pip install 'Shapely==1.7.1'
!pip install 'pyproj==3.0.0.post1'
!pip install folium

     |████████████████████████████████| 14.8 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.2 MB/s  eta 0:00:01


In [3]:
import boto3
from branca.colormap import linear
import fiona
import folium
import pandas as pd
import numpy as np
from pyproj import Proj, transform


import warnings
warnings.filterwarnings('ignore')

# Data Sources

Data for each italian region was downloaded from https://www.istat.it/it/archivio/104317 and saved to my personal S3. The data refers to 2011 census. 

I download data at individual "cella censuria" level, this is the most granular piece of geographical information made available within an administrative area. Those are usually towns and villages. Which in turn are part of on the 20 italian regions. 

Istat mades this data available in a set of files for each region. Therefore for each of those I downloaded:

1. {region_code}_indicatori_2011_sezioni.csv'. Which are the "celle censuarie" for each region + a set of features.
2. '{region_code}_11_WGS84.zip. The shapefiles in WGS84 format

# Functions

In [4]:

def wsg_to_latlon(x1, x2, inproj=32632):
    """Convert coordinates from WSG85 
    projection to latitude and longitude.
    
    Note: default inproj is what is used by 
    Italian Office of National Statistics (ISTAT) 
    ref: https://www.istat.it/it/files//2013/11/Descrizione-dati-Pubblicazione-2016.03.09.pdf
    
    Arguments:
        x1: float
        x2: float
        inproj: float
        
        https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python
    """
    inProj = Proj('epsg:{}'.format(inproj))
    outProj = Proj('epsg:4326')
    
    lat, long = transform(inProj,outProj,x1,y1)
    
    return lat, long


def shapefiles_to_latlong(sezioni):
    """Convert a shapefile in WSG84 projection 
    to latitud and longitude
    
    Arguments:
        sezioni: Pandas DF with a column "geometry"
        
    Return: 
        lat_long_geoms:list of shapefiles
        lat_long_centroids: list of tuples
    
    """

    lat_long_geoms = []
    lat_long_centroids = []

    for i in sezioni.itertuples():

        if i.Index % 1000 == 0:
            print(i.Index)

        geom = i.geometry

        if i.geometry['type'] == 'Polygon':

            coords = geom['coordinates'][0]

            # Convert WSG84 proj to lat-long
            inProj = Proj(init='epsg:32632')
            outProj = Proj(init='epsg:4326')
            wsg_x0 = [x[0] for x in coords]
            wsg_x1 = [x[1] for x in coords]
            lat, long = transform(inProj, outProj, wsg_x0, wsg_x1)

            lat_centroid = np.mean(lat)
            long_centroid = np.mean(long)

            # Create new set of coordinates
            new_coords = list(zip(lat, long))

            # Create New geometry based on latitude longitude
            new_geom = geom.copy()
            new_geom['coordinates'] = [new_coords]
            lat_long_geoms.append(new_geom)

            lat_long_centroids.append((lat_centroid, long_centroid))

        elif i.geometry['type'] == 'MultiPolygon':

            coords = geom['coordinates'][0][0]

            # Convert WSG84 proj to lat-long
            inProj = Proj(init='epsg:32632')
            outProj = Proj(init='epsg:4326')
            wsg_x0 = [x[0] for x in coords]
            wsg_x1 = [x[1] for x in coords]
            lat, long = transform(inProj, outProj, wsg_x0, wsg_x1)

            lat_centroid = np.mean(lat)
            long_centroid = np.mean(lat)

            # Create new set of coordinates
            new_coords = list(zip(lat, long))

            # Create New geometry based on latitude longitude
            new_geom = geom.copy()
            new_geom['coordinates'] = [[new_coords]]
            lat_long_geoms.append(new_geom)

            lat_long_centroids.append((lat_centroid, long_centroid))
            
    return lat_long_geoms, lat_long_centroids

    

In [5]:
# Get Objects in istat-sezioni folder

client = boto3.client('s3')
bucket = 'istat-sezioni'

resp = client.list_objects(Bucket = bucket)

objects = []

for i in resp['Contents']:
    
    if i['Size'] > 0:
        objects.append(i['Key'])
        

In [71]:
regions = ['R01', 'R02', 'R03', 'R04', 'R05',
           'R06', 'R07', 'R08', 'R09', 'R10',
           'R11', 'R12', 'R13', 'R14', 'R15',
           'R16', 'R17', 'R18', 'R19', 'R20',]


s3 = boto3.resource('s3')

for region in regions:
    
    print('Processing Region: ', region)
    
    feats_file = '2011/{}_indicatori_2011_sezioni.csv'.format(region)
    shapefile = '2011/{}_11_WGS84.zip'.format(region)
    
    # Download features file
    print('Downloading Features')
    local_file = feats_file.split("/")[-1]
    s3.meta.client.download_file(bucket, feats_file, local_file)

    sezioni = pd.read_csv(local_file, 
                   error_bad_lines = True, 
                   warn_bad_lines = True,
                   encoding = 'iso-8859-1',
                   sep = ";")
    
    # Download Shapefiles
    print('Downloading Shapefiles')
    local_shapefile = shapefile.split("/")[-1]
    s3.meta.client.download_file(bucket, shapefile, local_shapefile)
    
    local_shapefile_folder = local_shapefile.split(".")[0]
    
    # Unzip using a Bash command
    !unzip {local_shapefile}
    %cd {local_shapefile_folder}
    
    shapes = fiona.open("{}.shp".format(local_shapefile_folder))
    
    # Convert shapes to a dataframe and combine with features
    shapes_df = pd.DataFrame(shapes)
    shapes_df['SEZ2011'] = [x['properties']['SEZ2011'] for x in shapes]
    sezioni = pd.merge(sezioni, shapes_df, on = 'SEZ2011', how = 'left')
    
    # Remove files from local filesystem
    %cd ..
    !rm -rf {local_shapefile_folder}
    !rm -rf {local_shapefile}
    !rm -rf {local_file}
    
    # Convert Geometries
    print('Start WSG to Lat Long geometry conversion')
    lat_long_geoms, lat_long_centroids = shapefiles_to_latlong(sezioni)
    sezioni['geometry_ll'] = lat_long_geoms
    sezioni['centroid_ll'] = lat_long_centroids
    
    # Save Output
    sezioni.to_pickle('out/{}_istat_census.pickle'.format(region))
    sezioni.drop('geometry_ll', 1).to_csv(
        'out/{}_istat_census.csv'.format(region), index = False)
    
    print('End WSG to Lat Long geometry conversion')
    print("---------------------------------------")

# Save Data to S3

In [8]:
import os
import boto3
import pandas as pd

outdir = 'out'
outfiles = os.listdir(outdir)

In [68]:
csv_files = list(filter(lambda x: x.find('.csv') != -1, outfiles))

bucket = "istat-sezioni"

temps = []

for f in csv_files:
    print(f)
    
    temp = pd.read_csv(os.path.join(outdir, f))
    temp['centoid_lat'] = [float(x.split(',')[0].split('(')[1]) for x in temp['centroid_ll']]
    temp['centoid_long'] = [float(x.split(',')[1].split(')')[0].lstrip()) for x in temp['centroid_ll']]
    temp = temp.drop(['geometry', 'properties', 'centroid_ll'], 1)
    
    temps.append(temp)


fdf = pd.concat(temps)

R16_istat_census.csv
R09_istat_census.csv
R19_istat_census.csv
R17_istat_census.csv
R07_istat_census.csv
R10_istat_census.csv
R06_istat_census.csv
R02_istat_census.csv
R05_istat_census.csv
R11_istat_census.csv
R08_istat_census.csv
R15_istat_census.csv
R20_istat_census.csv
R04_istat_census.csv
R12_istat_census.csv
R18_istat_census.csv
R03_istat_census.csv
R14_istat_census.csv
R13_istat_census.csv
R01_istat_census.csv


In [69]:
# Save file sezioni
fname = 'sezioni-istat-2011.csv'

fdf.to_csv(fname, index=False)

s3_client = boto3.client('s3')
s3_client.upload_file(fname, 
                      bucket, 
                      'out/sezioni_istat/{}'.format(fname))

In [18]:
# Save pickeled files
csv_files = list(filter(lambda x: x.find('.pickle') != -1, outfiles))

bucket = "istat-sezioni"

for f in csv_files:
    print(f)
    s3_client = boto3.client('s3')
    s3_client.upload_file(os.path.join(outdir, f), 
                          bucket, 
                          'out/pickle/{}'.format(f))

R14_istat_census.pickle
R15_istat_census.pickle
R13_istat_census.pickle
R20_istat_census.pickle
R08_istat_census.pickle
R06_istat_census.pickle
R19_istat_census.pickle
R16_istat_census.pickle
R10_istat_census.pickle
R02_istat_census.pickle
R03_istat_census.pickle
R05_istat_census.pickle
R07_istat_census.pickle
R17_istat_census.pickle
R12_istat_census.pickle
R01_istat_census.pickle
R18_istat_census.pickle
R04_istat_census.pickle
R09_istat_census.pickle
R11_istat_census.pickle


In [43]:
# Save variables definition

fname = 'docs/tracciato_2011_sezioni.csv'

schema = pd.read_csv(fname, sep = ';', encoding = 'iso-8859-1')
schema.to_csv('tracciato_2011_sezioni.csv', encoding = 'utf-8', index = False)

s3_client = boto3.client('s3')
s3_client.upload_file('tracciato_2011_sezioni.csv', 
                      bucket, 
                      'out/schema_sezioni/tracciato_2011_sezioni.csv'.format(fname))

# Plot some data

In [11]:
sezioni = pd.read_pickle('sezioni.pickle')

In [13]:
color_variable = 'P1'
area = 'Cardito'

sub = sezioni[sezioni['COMUNE'].str.contains(area)].copy()

color_dict = sub[['SEZ2011', color_variable]]
colormap = linear.YlGn_09.scale(
    color_dict[color_variable].min(), color_dict[color_variable].max()
)

color_dict = color_dict.set_index('SEZ2011')['P1'].to_dict()
color_dict = {key: colormap(color_dict[key]) for key in color_dict.keys()}

In [14]:
sub = sezioni[sezioni['COMUNE'].str.contains('Cardito')]

cc = sub.iloc[0]['centroid_ll']

In [15]:


sub = sezioni[sezioni['COMUNE'].str.contains('Cardito')]

cc = sub.iloc[0]['centroid_ll']
centroid = (cc[1], cc[0])

m = folium.Map(centroid, zoom_start=12)

for cel in sub.itertuples():
    
    folium.GeoJson(
        cel.geometry_ll
    ).add_to(m)


m

In [ ]:
 {
        "fillColor": "green"
        if "e" in feature["properties"]["name"].lower()
        else "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }

In [16]:
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name="unemployment",
    style_function=lambda feature: {
        "fillColor": colormap(unemployment_dict[feature["id"]]),
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
).add_to(m)

folium.LayerControl().add_to(m)

m

NameError: name 'geo_json_data' is not defined

In [102]:
geoj = cel.geometry_ll
geoj['id'] = cel.SEZ2011